### Etapas
1. Criação de features
2. Tratamento de outliers
3. Normalização dos dados
4. Pipeline de pré-processamento
5. Separação em treino/teste
7. Treinamento do modelo

In [14]:
# @title Importação das bibliotecas utilizadas no programa

import pandas as pd
import numpy as np

from sklearn.base import BaseEstimator, TransformerMixin

# Carregamento do dataset
import requests
from pathlib import Path

# Criação de features
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors

# Normalização dos dados
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_selector

# Pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

# Treinamento do modelo 
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import cross_validate
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier, StackingClassifier,  AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import  AdaBoostClassifier, HistGradientBoostingClassifier
from sklearn.linear_model import RidgeClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

In [15]:
# @title Carregamento do dataset

# Versão Google Collab
'''
github_link = "https://github.com/mfigueireddo/ciencia-de-dados/blob/ba579573c5b8a9246ca04f7da29bc2c74c8b362c/datasets/pre-pipeline_wildfires.parquet"
url = github_link.replace("/blob/", "/raw/")

local_file_path = Path("/content/raw_wildfires.parquet")

# Faz uma requisição HTTP GET ao GitHub
with requests.get(url, stream=True) as request:

    request.raise_for_status() # Confere se houve êxito

    with open(local_file_path , "wb") as file:
        for chunk in request.iter_content(chunk_size=1024*1024):
            if chunk:
                file.write(chunk)

wildfires = pd.read_parquet(local_file_path,  engine="pyarrow") # Leitura realizada com a engine pyarrow
'''

# Versão VSCode
file_path = "../datasets/pre-pipeline_wildfires.parquet"

wildfires = pd.read_parquet(
    file_path,
    engine="pyarrow",
    use_nullable_dtypes=False
)

C:\Users\mathe\AppData\Local\Temp\ipykernel_18472\4240293704.py:26: FutureWarning: The argument 'use_nullable_dtypes' is deprecated and will be removed in a future version.
  wildfires = pd.read_parquet(


In [16]:
# @title Variáveis globais

data_column_name = 'data'
id_column_name = 'fire_id'
latitude_column_name = 'latitude'
longitude_column_name = 'longitude'
precipitation_column_name = 'precipitacao'
max_temperature_column_name = 'temperatura_max'
precipitation_sum_window_column_name = 'soma_precipitacao_90_dias'
max_temperature_mean_column_name = 'media_temp_max_90_dias'
season_column_name = 'estacao_ano_id'
region_column_name = 'regiao_incendio'
target_column_name = 'houve_incendio'

### 1. Criação de features
<u>**Features criadas**</u>
- Estação do ano
- Região do incêndio
- Soma da precipitação nos últimos 90 dias
- Média de temperatura máxima nos últimos 90 dias

**Observação 1**: as features estação do ano baseada e região do incêndio no momento não estão sendo utilizadas para treinar o modelo<br>
**Observação 2**: caso essas duas features passem a ser utilizadas no treinamento do modelo, elas deverão ser tratadas porque são categóricas e não há relação numérica entre seus valores.

In [17]:
# @title Classe FeatureCreation

class FeaturesCreation(BaseEstimator, TransformerMixin):

    def __init__(self):
        # Colunas utilizadas
        self.m_date_column = data_column_name
        self.m_group_column = id_column_name
        self.m_latitude_column = latitude_column_name
        self.m_longitude_column = longitude_column_name
        self.m_precipitation_column = precipitation_column_name 
        self.m_max_temperature_column = max_temperature_column_name 

        # Parâmetros personalizados para criação das features
        self.m_precipitation_window_days = 90
        self.m_max_temperature_window_days = 90

        # Parâmetros do DBSCAN
        self.mm_max_radiuskm = 1.0
        self.m_min_samples = 5

        # Objetos aprendidos no fit
        self.m_dbscan = None
        self.m_nearest_neighbors_core = None
        self.m_core_labels = None
        self.m_max_radius = None

    # Conversão necessária para o DBSCAN
    @staticmethod
    def convert_to_radians(latitude_or_longitude):
        return np.radians(latitude_or_longitude.astype(float))
    
    @staticmethod
    def convert_month_to_season(month):
        if month in (12, 1, 2): return 0  # Inverno
        if month in (3, 4, 5): return 1  # Primavera
        if month in (6, 7, 8): return 2  # Verão
        return 3  # 9, 10, 11 -> Outono

    def fit(self, dataframe, target=None):
        dataframe = dataframe.copy()

        # Desativa o DBSCAN caso não haja latitude e longitude
        missing_cols = [column for column in [self.m_latitude_column, self.m_longitude_column] if column not in dataframe.columns]
        if missing_cols:
            self.m_dbscan = None
            self.m_nearest_neighbors_core = None
            self.m_core_labels = None
            self.m_max_radius = None
            return self

        latitude_or_longitude = dataframe[[self.m_latitude_column, self.m_longitude_column]].to_numpy()
        latitude_or_longitude_radians = self.convert_to_radians(latitude_or_longitude)

        earth_radius = 6371.0
        self.m_max_radius = self.mm_max_radiuskm / earth_radius

        dbscan = DBSCAN(eps=self.m_max_radius, min_samples=self.m_min_samples, metric='haversine')
        dbscan.fit(latitude_or_longitude_radians)
        self.m_dbscan = dbscan

        # Treina um NearestNeighbors apenas nos pontos-core para atribuição de novos pontos à clusters existentes em transform()
        core_mask = np.zeros_like(dbscan.labels_, dtype=bool)
        if hasattr(dbscan, 'core_sample_indices_') and len(dbscan.core_sample_indices_) > 0:
            core_mask[dbscan.core_sample_indices_] = True
            core_points = latitude_or_longitude_radians[core_mask]
            core_labels = dbscan.labels_[core_mask]

            if len(core_points) > 0:
                nearest_neighbors = NearestNeighbors(n_neighbors=1, metric='haversine')
                nearest_neighbors.fit(core_points)
                self.m_nearest_neighbors_core = nearest_neighbors
                self.m_core_labels = core_labels
            else:
                self.m_nearest_neighbors_core = None
                self.m_core_labels = None
        else:
            self.m_nearest_neighbors_core = None
            self.m_core_labels = None

        return self

    # Rotula novos pontos
    def assign_dbscanlabels(self, dataframe):

        # Se não tivemos lat/lon ou DBSCAN treinado, devolve NaN
        if self.m_nearest_neighbors_core is None or self.m_core_labels is None or self.m_max_radius is None:
            return pd.Series([-1] * len(dataframe), index=dataframe.index, dtype='int64')

        latitude_or_longitude = dataframe[[self.m_latitude_column, self.m_longitude_column]].to_numpy()
        latitude_or_longitude_radians = self.convert_to_radians(latitude_or_longitude)

        # Atribui rótulo do core mais próximo, desde que dentro do raio
        distances, indices = self.m_nearest_neighbors_core.kneighbors(latitude_or_longitude_radians, n_neighbors=1, return_distance=True)
        distances = distances.reshape(-1)
        indices = indices.reshape(-1)

        labels = np.full(len(dataframe), -1, dtype='int64')
        within = distances <= self.m_max_radius
        labels[within] = self.m_core_labels[indices[within]]

        return pd.Series(labels, index=dataframe.index, dtype='int64')

    # Adiciona médias móveis e soma pro grupo de incêndio
    def add_temporal_rollings(self, dataframe):
        
        # Ordena por grupo e tempo para garantir rolling correto
        if self.m_group_column in dataframe.columns and self.m_date_column in dataframe.columns:
            dataframe = dataframe.sort_values([self.m_group_column, self.m_date_column])
        else:
            # Se faltar algo, só ordena por data (se existir)
            if self.m_date_column in dataframe.columns:
                dataframe = dataframe.sort_values(self.m_date_column)

        # Rolling de precipitação (soma dos últimos 14 dias)
        if self.m_precipitation_column in dataframe.columns:
            dataframe[precipitation_sum_window_column_name] = (
                dataframe.groupby(self.m_group_column, dropna=False)[self.m_precipitation_column]
                  .rolling(self.m_precipitation_window_days, min_periods=1)
                  .sum()
                  .reset_index(level=0, drop=True)
            )
        else:
            dataframe[precipitation_sum_window_column_name] = np.nan

        # Rolling de temperatura máxima (média dos últimos 7 dias)
        if self.m_max_temperature_column in dataframe.columns:
            dataframe[max_temperature_mean_column_name] = (
                dataframe.groupby(self.m_group_column, dropna=False)[self.m_max_temperature_column]
                  .rolling(self.m_max_temperature_window_days, min_periods=1)
                  .mean()
                  .reset_index(level=0, drop=True)
            )
        else:
            dataframe[max_temperature_mean_column_name] = np.nan

        return dataframe

    # Aplica transformações e cria as novas features
    def transform(self, dataframe, target=None):
        
        # Trabalha em DataFrame para manter nomes/índices
        dataframe = pd.DataFrame(dataframe).copy()

        # Estação do ano
        if self.m_date_column in dataframe.columns:
            # Garante dtype datetime
            dataframe[self.m_date_column] = pd.to_datetime(dataframe[self.m_date_column], errors='coerce')
            estacao = dataframe[self.m_date_column].dt.month.map(self.convert_month_to_season).astype('Int64')
            dataframe[season_column_name] = estacao.astype('float').astype('Int64')  # evita problemas de NaN -> imputar depois
            dataframe[season_column_name] = dataframe[season_column_name].astype('float')
        else:
            dataframe[season_column_name] = np.nan

        # Região 
        if all(c in dataframe.columns for c in [self.m_latitude_column, self.m_longitude_column]):
            dataframe[region_column_name] = self.assign_dbscanlabels(dataframe).astype('int64')
        else:
            dataframe[region_column_name] = -1

        # Rollings temporais
        dataframe = self.add_temporal_rollings(dataframe)

        '''
        for col, dtype in dataframe.dtypes.items():
            print(f" - {col}: {dtype}")
        '''

        return dataframe


### 2. Tratamento de outliers

**Transformação Logarítmica**
- Aplica log(x) ou log(x+constante) para valores positivos
- Muito eficaz para dados com distribuição assimétrica positiva
- Comprime valores grandes e expande valores pequenos
- Fórmula: X_log = log(X + c), onde c evita log(0)

**Transformação Raiz Quadrada**
- Menos drástica que a transformação logarítmica
- Útil para dados de contagem e variáveis positivamente assimétricas
- Fórmula: X_sqrt = sqrt(X)

**Winsorização (Capping/Clipping)**

A **Winsorização** é uma técnica de tratamento de outliers que **limita valores extremos** sem removê-los completamente. Em vez de excluir outliers, substituímos os valores extremos pelos valores de percentis específicos.

**Como funciona:**
- Define-se limites baseados em percentis (ex: 5º e 95º percentil)
- Valores abaixo do limite inferior são substituídos pelo valor do limite inferior
- Valores acima do limite superior são substituídos pelo valor do limite superior

| Variável                                 | Melhor método          | Resultado obtido |
| :--------------------------------------- | :--------------------  | :-------------------------------------------------------------------------------------------------------- |
| **precipitacao**                         | **Log(x + 1)**         | Assimetria (7.87 → 2.59)                                                                                  |
| **umidade_relativa_max**                 | **Winsorização**       | Outliers (23 → 0)                                                                                         |
| **umidade_relativa_min**                 | **Winsorização**       | Outliers (1155 → 0)                                                                                       |
| **umidade_especifica**                   | **Sqrt**               | Simetria (0.89 → 0.16) / Outliers (6967 → 2102)                                                           |
| **radiacao_solar**                       | **Sem transformação**  |                                                                                                           |
| **temperatura_min**                      | **Winsorização**       | Outliers (5066 → 0)                                                                                       |
| **temperatura_max**                      | **Winsorização**       | Outliers (5066 → 0)                                                                                       |
| **velocidade_vento**                     | **Log(x + 1)**         | Assimetria (1.23 → 0.19) / Outliers (8723 → 1128)                                                         |
| **indice_queima**                        | **Winsorização**       |                                                                                                           |
| **umidade_combustivel_morto_100_horas**  | **Sqrt**               | Outliers (44 → 9)                                                                                         |
| **umidade_combustivel_morto_1000_horas** | **Sqrt**               | Outliers (373 → 4)                                                                                        |
| **componente_energia_lancada**           | **Sem transformação**  |                                                                                                           |
| **evapotranspiracao_real**               | **Sqrt**               | Assimetria (0.71 → -0.00) / Outliers (3292 → 153)                                                         |
| **evapotranspiracao_potencial**          | **Log(x + 1)**         | Outliers (679 → 0)                                                                                        |
| **deficit_pressao_vapor**                | **Log(x + 1)**         | Outliers (14758 → 672)                                                                                    |


In [18]:
# @title Classe OutliersTreatment

class OutliersTreatment(BaseEstimator, TransformerMixin):

    def __init__(self):
        self.m_log_columns = [
            "precipitacao",
            "velocidade_vento",
            "evapotranspiracao_potencial",
            "deficit_pressao_vapor",
        ]
        self.m_sqrt_columns = [
            "umidade_especifica",
            "umidade_combustivel_morto_100_horas",
            "umidade_combustivel_morto_1000_horas",
            "evapotranspiracao_real",
        ]
        self.m_winsor_columns = [
            "umidade_relativa_max",
            "umidade_relativa_min",
            "temperatura_min",
            "temperatura_max",
            "indice_queima",
        ]
        self.m_winsor_limits = (0.05, 0.05)

    # Calcula parâmetros necessários para aplicar as transformações corretamente
    def fit(self, dataframe, target=None):

        # Garante que o usuário esteja enviado um dataframe no formato correto
        dataframe = dataframe if isinstance(dataframe, pd.DataFrame) else pd.DataFrame(dataframe)

        # Cálculo de offsets para garantir que não haverão valores zerados ou negativos
        # "coerce" convete valores inválidos para NaN

        self.m_log_offset = {}
        for column in self.m_log_columns:
            if column in dataframe.columns:
                series = pd.to_numeric(dataframe[column], errors="coerce")
                min_value = series.min()
                self.m_log_offset[column] = (abs(min_value) + 1) if pd.notna(min_value) and min_value <= 0 else 1.0

        self.m_sqrt_offset = {}
        for column in self.m_sqrt_columns:
            if column in dataframe.columns:
                series = pd.to_numeric(dataframe[column], errors="coerce")
                min_value = series.min()
                self.m_sqrt_offset[column] = (abs(min_value) + 0.01) if pd.notna(min_value) and min_value < 0 else 0.0

        # Garante que a winsorização só seja feita com colunas que realmente estão no dataframe
        actual_columns_to_winsor = [column for column in self.m_winsor_columns if column in dataframe.columns]
        low_quantile, high_quantile = self.m_winsor_limits

        if actual_columns_to_winsor:
            # Converte cada coluna para numérico (coerces -> NaN) e calcula quantis por coluna
            winsor_dataframe = dataframe[actual_columns_to_winsor].apply(pd.to_numeric, errors="coerce")
            self.m_low_quantile  = winsor_dataframe.quantile(low_quantile)
            self.m_high_quantile = winsor_dataframe.quantile(1 - high_quantile)
        else:
            # garante atributos vazios para não quebrar no transform()
            self.m_low_quantile  = pd.Series(dtype=float)
            self.m_high_quantile = pd.Series(dtype=float)

        return self

    # Aplica as transformações
    def transform(self, dataframe):

        # Garante que o usuário esteja enviado o dataframe correto
        dataframe = dataframe.copy() if isinstance(dataframe, pd.DataFrame) else pd.DataFrame(dataframe).copy()

        # LOG
        for column, offset in self.m_log_offset.items():
            if column in dataframe.columns:
                series = pd.to_numeric(dataframe[column], errors="coerce")
                dataframe[column] = np.log(series + offset)

        # SQRT
        for column, offset in self.m_sqrt_offset.items():
            if column in dataframe.columns:
                series = pd.to_numeric(dataframe[column], errors="coerce")
                dataframe[column] = np.sqrt(series + offset)

        # Winsorização
        for column in self.m_low_quantile.index:
            if column in dataframe.columns:
                series = pd.to_numeric(dataframe[column], errors="coerce")
                dataframe[column] = series.clip(lower=self.m_low_quantile[column], upper=self.m_high_quantile[column])

        return dataframe

### 3. Normalização dos dados

São utilizados para normalizar os dados numéricos
- SimpleImputer
- MinMaxScaler

No momento, não estão sendo utilizadas features categóricas para treinar o modelo. Contudo, existe a implementação de uma normalização provisória para as mesmas utilizando
- SimpleImputer
- OneHotEncoder

In [19]:
# @title Classe DataNormalization

'''
categoric_cols = [season_column_name, region_column_name] 

def numeric_columns_selector(dataframe): 
    # Seleciona colunas numéricas, exceto as categóricas codificadas numericamente
    num = dataframe.select_dtypes(include='number').columns.tolist() 
    return [column for column in num if column not in categoric_cols] 

categoric_columns_pipeline = Pipeline(steps=[ 
    ('imputer', SimpleImputer(strategy='most_frequent')), 
    ('one_hot_encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False)), 
])
'''

numeric_columns_pipeline = Pipeline(steps=[ 
    ('imputer', SimpleImputer(strategy='median')), 
    ('scaler', MinMaxScaler()), 
    ]) 

DataNormalization = ColumnTransformer(
    transformers=[
        ('numerical', numeric_columns_pipeline, make_column_selector(dtype_include='number')),
        #('categoric', categoric_columns_pipeline, categoric_cols)
    ],
    remainder='passthrough'  # mantém quaisquer colunas não listadas
)

### 4. Pipeline de pré-processamento

Concentra 5 passos
- Criação de features
- Tratamento de outliers
- Eliminação de features indesejadas no treinamento dos modelos
- Normalização dos dados
- Sanitização dos dados (garantindo que estão no formato esperado)


In [20]:
# @title Classe ColumnDropper

# Elimina colunas não desejáveis no treinamento do modelo
class ColumnDropper(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.columns = [data_column_name, id_column_name, latitude_column_name, longitude_column_name, season_column_name, region_column_name]
    def fit(self, dataframe, target=None):
        return self
    def transform(self, dataframe):
        return dataframe.drop(columns=self.columns, errors="ignore")

In [21]:
# @title Função sanitizer

# Remove data e fire_id, além de converter o dataframe para o formato esperado pelos classificadores
def sanitizer(features):
    # Transforma em numpy.ndarray
    if isinstance(features, pd.DataFrame):
        features = features.select_dtypes(include='number')  
    return features   

In [22]:
# @title Pipeline preprocess

preprocess = Pipeline(steps=[
    ("features_creation", FeaturesCreation()),
    ("outliers_treatment", OutliersTreatment()),
    ("drop_unused_columns", ColumnDropper()),
    ("data_normalization", DataNormalization),
    ("sanitize", FunctionTransformer(sanitizer, validate=False, feature_names_out='one-to-one'))
])

### 5. Separação em treino/teste

**Time Series Cross Validation**

A Time Series Cross Validation é uma técnica especializada para validar modelos quando os dados possuem ordem cronológica. Diferente das técnicas tradicionais, ela respeita a estrutura temporal dos dados.

Foi implemetada uma função personalizada para que dados referentes ao mesmo incêndio permanecessem nos mesmos grupos

In [23]:
# @title Função group_time_series_cross_validation

# Gera folds (train_idx, test_idx) respeitando ordem temporal por grupo, embargo em nível de grupo e exclusão mútua treino/teste por grupo.
def group_time_series_cross_validation():
    dataframe = wildfires
    time_column = data_column_name
    group_column = id_column_name
    folds_amount = 5
    fold_groups_size = 1
    gap_between_groups_amount = 0

    # Ordena grupos pelo primeiro timestamp
    first_time = (
        dataframe[[group_column, time_column]]
        .dropna(subset=[time_column])
        .groupby(group_column)[time_column]
        .min()
        .sort_values()
    )
    ordered_groups = first_time.index.to_numpy()
    ordered_groups_len = len(ordered_groups)

    groups_amount_by_step = fold_groups_size

    min_train_groups = max(1, fold_groups_size) # pelo menos 1

    # Âncora: último grupo incluso no treino
    # Precisamos garantir espaço para gap + teste à frente
    max_anchor = ordered_groups_len - gap_between_groups_amount - fold_groups_size
    if max_anchor <= min_train_groups:
        return  # não há splits possíveis

    splits = 0
    anchor = min_train_groups
    while anchor <= max_anchor and splits < folds_amount:
        train_groups = ordered_groups[:anchor]

        test_start = anchor + gap_between_groups_amount
        test_end = test_start + fold_groups_size
        test_groups = ordered_groups[test_start:test_end]

        train_idx = dataframe.index[dataframe[group_column].isin(train_groups)].to_numpy()
        test_idx  = dataframe.index[dataframe[group_column].isin(test_groups)].to_numpy()

        if train_idx.size and test_idx.size:
            yield (train_idx, test_idx)
            splits += 1

        anchor += groups_amount_by_step

cross_validation = cross_validation_splits = list(group_time_series_cross_validation())


### 6. Treinamento dos modelos

Os modelos treinados foram
- Dummy (mais frequente)
- Regressão Logística
- Árvore de Decisão
- Random Forest
- Naive Bayes
- KNN
- Gradient Boosting **(NOVO)**
- AdaBoost **(NOVO)**
- HistGradientBoosting **(NOVO)**
- RidgeClassifier **(NOVO)**
- XGBoost **(NOVO)**
- CatBoost **(NOVO)**
- LightGBM **(NOVO)**

In [45]:
# @title Escolha dos modelos e das métricas

modelos = {
    "Dummy (mais frequente)": DummyClassifier(strategy="most_frequent"), # Baseline simples
    "Regressão Logística": LogisticRegression(C=0.5, penalty='l2', solver='liblinear', max_iter=2000), # Regularização L2 leve
    "Árvore de Decisão": DecisionTreeClassifier(max_depth=8, min_samples_split=4, min_samples_leaf=2, random_state=42), # Controle de profundidade e tamanho da folha
    "Random Forest": RandomForestClassifier(n_estimators=150, max_depth=10, min_samples_split=5,random_state=42, n_jobs=-1), # Mais árvores e profundidade moderada
    "Naive Bayes": GaussianNB(var_smoothing=1e-8), # Suavização leve (mais estável)
    "KNN": KNeighborsClassifier(n_neighbors=7, weights='distance'), # Mais vizinhos e distância ponderada
    "Gradient Boosting": GradientBoostingClassifier(n_estimators=100, learning_rate=0.05, max_depth=5, random_state=42), # Parâmetros leves conforme aula
    "AdaBoost": AdaBoostClassifier(n_estimators=100, learning_rate=0.5, random_state=42), # Taxa de aprendizado menor
    "HistGradientBoosting": HistGradientBoostingClassifier(max_iter=150, learning_rate=0.05, max_depth=5, random_state=42), # Mais iterações e taxa menor
    "RidgeClassifier": RidgeClassifier(alpha=1.0), # Regularização leve
    "XGBoost": XGBClassifier(eval_metric='logloss', n_estimators=150, learning_rate=0.05, max_depth=5, subsample=0.8, colsample_bytree=0.8, random_state=42), #  Parâmetros típicos de equilíbrio (aula 15 parte 4)
    "CatBoost": CatBoostClassifier(iterations=150, learning_rate=0.05, depth=5,verbose=0, random_state=42), # Taxa de aprendizado reduzida e iterações extras
    # "LightGBM": LGBMClassifier(n_estimators=150, learning_rate=0.05, num_leaves=31, max_depth=6, random_state=42) # Profundidade controlada e taxa moderada
}

# Métricas de avaliação
scoring = {
    'accuracy': 'accuracy',
    'precision': make_scorer(precision_score, zero_division=0),
    'recall': make_scorer(recall_score, zero_division=0),
    'f1': make_scorer(f1_score, zero_division=0),
    'roc_auc': 'roc_auc' 
}

In [46]:
# @title Algoritmo de treinamento

features = wildfires.drop(columns=target_column_name)
target = wildfires[target_column_name].astype(int)

resultados = []

for nome, modelo in modelos.items():
    print(f"Treinando modelo: {nome}...")

    try:
        pipeline = Pipeline([
            ("preprocess", preprocess),
            ("nan_shield", SimpleImputer(strategy="constant", fill_value=0.0)), # Blindagem contra NaN
            ("classificator", modelo)
        ])

        scores = cross_validate(pipeline, features, target, cv=cross_validation, scoring=scoring, n_jobs=1)

        resultados.append({
            "Modelo": nome,
            "Accuracy": np.mean(scores['test_accuracy']),
            "Precision": np.mean(scores['test_precision']),
            "Recall": np.mean(scores['test_recall']),
            "F1-score": np.mean(scores['test_f1']),
            "ROC AUC": np.mean(scores['test_roc_auc']),
        })

        print(f" Modelo {nome} treinado com sucesso.\n")

    except Exception as e:
        print(f" Erro ao rodar o modelo {nome}: {e}\n")

# Mostra os resultados
df_resultados = pd.DataFrame(resultados).sort_values("F1-score", ascending=False)
df_resultados

Treinando modelo: Dummy (mais frequente)...
 Modelo Dummy (mais frequente) treinado com sucesso.

Treinando modelo: Regressão Logística...
 Modelo Regressão Logística treinado com sucesso.

Treinando modelo: Árvore de Decisão...
 Modelo Árvore de Decisão treinado com sucesso.

Treinando modelo: Random Forest...
 Modelo Random Forest treinado com sucesso.

Treinando modelo: Naive Bayes...
 Modelo Naive Bayes treinado com sucesso.

Treinando modelo: KNN...
 Modelo KNN treinado com sucesso.

Treinando modelo: Gradient Boosting...
 Modelo Gradient Boosting treinado com sucesso.

Treinando modelo: AdaBoost...
 Modelo AdaBoost treinado com sucesso.

Treinando modelo: HistGradientBoosting...
 Modelo HistGradientBoosting treinado com sucesso.

Treinando modelo: RidgeClassifier...
 Modelo RidgeClassifier treinado com sucesso.

Treinando modelo: XGBoost...
 Modelo XGBoost treinado com sucesso.

Treinando modelo: CatBoost...
 Modelo CatBoost treinado com sucesso.



,Modelo,Accuracy,Precision,Recall,F1-score,ROC AUC
10,XGBoost,0.934667,0.830435,0.653333,0.635167,0.949630
4,Naive Bayes,0.908000,0.724856,0.653333,0.570613,0.953679
8,HistGradientBoosting,0.922667,0.628235,0.560000,0.516660,0.968049
5,KNN,0.926667,0.900000,0.453333,0.511077,0.873333
3,Random Forest,0.925333,0.859274,0.506667,0.497508,0.946963
11,CatBoost,0.906667,0.779464,0.560000,0.493215,0.946568
7,AdaBoost,0.905333,0.773950,0.573333,0.486567,0.895358
6,Gradient Boosting,0.921333,0.581263,0.493333,0.466078,0.830222
2,Árvore de Decisão,0.909333,0.506298,0.413333,0.390672,0.727062
9,RidgeClassifier,0.922667,0.520000,0.360000,0.357059,0.993284
